# Дискретные системы

С широким распространением цифровых вычислительных устройств дискретные системы управления стали занимать доминирующее положение в наборе инструментов автоматизации. Объясняется это довольно просто: реализация большинства систем управления на различного рода программируемых контроллерах значительно проще, чем на аналоговых компонентах. Кроме того, цифровое исполнение исключает "дрейф" параметров регулятора в течении времени и в зависимости от услови окружающей среды, в отличии от аналоговых систем, где стабильность параметров компонентов является серьезной проблемой.

Однако, очевидно, что описание дискретных систем с помощью дифференциальных уравнений невозможен, что не позволяет прямо преобразовывать передаточную функцию системы управления в программный код (и обратно) Поэтому, начиная с 60-х годов XX века начала активно формироваться теория управления для дискретных систем.

В этой лабораторной работе рассмотрено влияние дискретизации систем управления на качество управления на примере маятника на каретке.

В рамках лабораторной работы вам необходмо будет синтезировать дискретный регулятор на основе непрерывной модели объекта и исследовать влияние изменения шага дискретизации на показатели качества системы.

## Подготовка к работе

Введите значения своего варианта задания на практических работах.

In [0]:
variant = 0

В следующем блоке расчитываются параметры системы на основе номера варианта. Менять в нем ничего не следует, но необходимо выполнить.

In [0]:
# Длинна маятника
l = 0.5
m = 1
M = 0.5

# Ширина рельс
width = 4 * variant # N - номер варианта

print("Параметры системы:\n Длина маятника: {} \n Масса маятника: {} \n Масса каретки: {} \n Ширина рельс: {}".format(l,m,M,width))

Параметры системы:
 Длина маятника: 0.5 
 Масса маятника: 1 
 Масса каретки: 0.5 
 Ширина рельс: 0


## Объект управления
Следующий код реализует рассматриваемый объект управления с учетом дискретности регулятора.

Для пользователя основным интерфейсом является метод установки функции управления и запуска моделирования.

Основе записи уравнений динамики механической системы с помощью уравнений Лагранжа второго рода и подстановки кинетической и потенциальной энергии в уравнения получим математическую модель перевернутого маятника в виде системы двух дифференциальных уравнений второго порядка.

$$
  {d^2 θ\over dt^2} = {g \sin(θ) + \cos(θ) {d^2 x \over dt^2} \over l}+ {a\over ml^2} {dθ \over dt} \\
{d^2 x \over dt^2} = - {ml \over (M+m) }(\cos θ  {d^2 θ \over dt^2 } + \sin θ ({ dθ \over dt })^2 )- {c \over (M+m)} {dx \over dt}+f
$$

где $М$ — масса каретки, $x$ — положение каретки, $f$ — воздействие на каретку, $m$ — сосредоточенная масса маятника, $l$ — длина маятника, $\theta$ — угол отклонения маятника, $a$ — коэффициент вязкого трения маятника, $c$ — коэффициент вязкого трения каретки.

Уравнения представляют собой выражения баланса моментов, действующих на маятник, и баланса сил, действующих на каретку.

In [0]:
from scipy.integrate import odeint
import numpy as np

class ObjectModel:
    def __init__(self, params):
        self._var = variant
        self._ctrl_fcn = real_object._default_control
        l, m, M, width = params
        self._l = l
        self._m = m
        self._M = M
        self._plant_width = width
    
    def saturation(self, x, p1, p2):
        if x > p1:
            x = p1
        elif x < -p2:
            x = -p2
        return x
    
    def _ode(self, y, t, k):
        '''
        Функция принимает на вход вектор переменных состояния и реализует по сути систему в форме Коши
        x -- текущий вектор переменных состояния
        t -- текущее время
        k -- значения параметров
        '''
        #TODO: Реализация системы д.у.
        omega, theta, V, x = y
        u = self._get_u(x, t)
        
        dydt = (lin_par_1 * self.nonlin_fcns[self.nonlin_type](u, nonlin_par_1, nonlin_par_2) - x) / lin_par_2
        return dydt
    
    def _default_control(x, t):
        """
        Управление по умолчанию. Нулевой вход
        """
        return 0
    
    def _get_u(self, x, t):
        """
        Получить значение управления при значениях переменных состояния x и времени t
        """
        return self._ctrl_fcn(x, t)
    
    def set_u_fcn(self, new_u):
        """
        Установить новую функцию управления
        формат функции: fcn(x, t)
        """
        self._ctrl_fcn = new_u
    
    def calcODE(self, y0, ts=10, nt=1001):
        """
        Вспомогательная функция для получения решения систему ДУ, "Проведение эксперимента" с заданным воздействием
        """
        y0 = [y0,]
        t = np.linspace(0, ts, nt)
        args = (self._params, )
        sol = odeint(self._ode, y0, t, args)
        return sol, t